In [ ]:
# Step 0: Data preprocessing
---
In this step, we preprocess data from historical data or data generation.

### 1. Import the Necessary Packages
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy
from cycler import cycler

# # Set up matplotlib and seaborn styles
# plt.style.use('bmh')
# mpl.rcParams['axes.prop_cycle'] = cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

np.random.seed(0)  # For reproducibility
df_original_data = pd.read_csv('data/data.csv')
df_original_data.reset_index(inplace=True, drop=True)
df_original_data
# Find's boolean mask of non-null values, then sums them by row - only checking kWhs
non_null_counts_by_consumer = df_original_data.iloc[:, 2:].notnull().sum(axis=1)

# Print first five values to check
non_null_counts_by_consumer.head()

# Using a distribution plot to visualize non-null counts in data
plt.figure(figsize=(6, 3))
sns.histplot(non_null_counts_by_consumer, kde=False, edgecolor='k', linewidth=2)
plt.xlabel('Non-Null kWhs')
plt.ylabel('Frequency')
plt.title('Distribution of Non-Null kWhs Before `NaN` Replacement')
df_original_data.isnull().sum().sum()
# Step 2: Detect and remove rows with more than 600 NaNs
threshold = 600  # Set the threshold for NaN values
rows_to_drop = df_original_data[df_original_data.isnull().sum(axis=1) > threshold].index  # Get index of rows with more than 600 NaNs

# Step 3: Remove these rows from the DataFrame
df_original_data = df_original_data.drop(index=rows_to_drop)
df_original_data
df_original_data.isnull().sum().sum()
df_original_data.reset_index(inplace=True, drop=True)
# #droping duplicate row
# dropIndex = df_data[df_data.duplicated()].index  # duplicates drop
# df_data = df_data.drop(dropIndex, axis=0)   # droping duplicate value present wen two row are same
# df_info = df_info.drop(dropIndex, axis=0) # droping duplicate index infodata

# zeroIndex = df_data[(df_data.sum(axis=1) == 0)].index  # zero rows drop
# df_data = df_data.drop(zeroIndex, axis=0)
# infoData = df_info.drop(zeroIndex, axis=0)
df_info = pd.DataFrame()
df_info['CONS_NO'] = df_original_data['CONS_NO']
df_info['FLAG'] = df_original_data['FLAG']
df_original_data = df_original_data.drop(['FLAG', 'CONS_NO'], axis=1)   #axis 1 column ,axis 0 row
#change column name to dates(2014/1/1 to 2014-01-01)
df_original_data.columns = pd.to_datetime(df_original_data.columns)  #columns reindexing according to dates

#sort data accoding to date( as previusoly column are unsorted)
df_original_data = df_original_data.reindex(sorted(df_original_data.columns), axis=1)
# Calculating the mean and standard deviation
max_by_consumer = df_original_data.max(axis=1)
average_by_consumer = df_original_data.mean(axis=1)
std_by_consumer = df_original_data.std(axis=1)
df_original_data.mean().mean()
max_by_consumer.idxmax()
df_original_data.loc[5962].idxmax()
df_original_data.loc[5962].plot()
df_original_data.max().max(), df_original_data.mean().mean()
import time

# Attempting numpy array-based solution
kWhs = df_original_data.values.copy()

# Does it have the right shape
rows, cols = kWhs.shape

# Record time at beginning of loop
t_0 = time.time()

# For every consumer
for i in range(0, rows):
  # Print a dot as an update after every 400 consumers
  if (i % 400  == 0):
    print("#", end="")

  # And for every value of that consumer
  for j in range(1, cols - 1):
    # If the current value is undefined
    if (np.isnan(kWhs[i, j])):
      if (~np.isnan(kWhs[i, j - 1]) and ~np.isnan(kWhs[i, j + 1])):
        kWhs[i, j] = np.mean([kWhs[i, j - 1], kWhs[i, j + 1]])
      elif(np.isnan(kWhs[i, j - 1]) or np.isnan(kWhs[i, j + 1])):
        kWhs[i, j] = 0.0

# Record time at end of loop
t_f = time.time()
print('\n', (t_f - t_0)/60)
# Replacing kWh values with processed numpy array values in the dataframe
df_data_filled = df_original_data.copy(deep=True)
df_data_filled.iloc[:, 0:] = kWhs

# Remaining `NaN`s are in the first and last columns - replace with 0
df_data_filled.fillna(0, inplace=True)
df_data_filled
df_data_filled.isnull().sum().sum()
df_data_filled.max().max(), df_data_filled.mean().mean()
# Calculating the mean and standard deviation
max_by_consumer = df_data_filled.max(axis=1)
average_by_consumer = df_data_filled.mean(axis=1)
std_by_consumer = df_data_filled.std(axis=1)
df_data_filled.shape[0]
df_outliers_handled = df_data_filled.copy(deep=True)
for i in range(df_outliers_handled.shape[0]):
    mean = df_outliers_handled.loc[i].mean()
    sd = df_outliers_handled.loc[i].std()
    upper_limit = mean + 2 * sd
    lower_limit = mean - 2 * sd
    
    # Capping the data at the upper and lower sigma limits.
    df_outliers_handled.loc[i] = np.where(df_outliers_handled.loc[i] > upper_limit, upper_limit,
                                          np.where(df_outliers_handled.loc[i] < lower_limit, lower_limit, df_outliers_handled.loc[i]))

# for i in range(df_data_filled.shape[0]):
#     mean = df_data_filled.loc[i].mean()
#     sd = df_data_filled.loc[i].std()
#     arr = df_data_filled.loc[i].values

#     arr[arr > (mean + 2 * sd)] = mean + 2 * sd
#     df_data_filled.loc[i] = arr

# # Calculating the mean and standard deviation
# max_by_consumer = df_outliers_handled.max(axis=1)
# average_by_consumer = df_outliers_handled.mean(axis=1)
# std_by_consumer = df_outliers_handled.std(axis=1)

# average_by_consumer
# Calculating the mean and standard deviation
max_by_consumer = df_outliers_handled.max(axis=1)
average_by_consumer = df_outliers_handled.mean(axis=1)
std_by_consumer = df_outliers_handled.std(axis=1)
average_by_consumer
max_by_consumer
max_by_consumer.idxmax()
df_outliers_handled.loc[5388].plot()
df_info.loc[5388]
max(df_outliers_handled.loc[5388])
df_data_filled.max().max(), df_data_filled.mean().mean()
df_outliers_handled.max().max(), df_outliers_handled.mean().mean()
df_outliers_handled.isnull().sum().sum()
df_final = pd.concat([df_info, df_outliers_handled], axis=1)
df_final
df_final.to_csv("Data/df_final.csv")
# df_data_filled.to_csv("Data/new_clean_data.csv")
# df_info.to_csv("Data/new_info.csv")
df_final=pd.read_csv('data/df_final.csv')
df_final = df_final.drop(['Unnamed: 0'], axis=1)   #axis 1 column ,axis 0 row
df_final
odf = df_final.copy()
# SEPARATING DEPENDENT AND INDEPENDENT VARIABLES
x = odf.iloc[:, 2:].values
y = odf.iloc[:, 1].values
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)
from sklearn.decomposition import PCA

# Assuming you have your data in x and labels in y

# Plotting the distribution of classes in y
unique_classes, class_counts = np.unique(y, return_counts=True)
plt.bar(unique_classes, class_counts)
plt.xlabel('FLAG')
plt.ylabel('COUNT')
plt.title('DATASET DISTRIBUTION')
plt.show()

# Performing PCA to reduce the dimensionality of x for visualization
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x)

# Plotting the reduced x in 2D
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y, cmap='viridis', alpha=0.5)
plt.xlabel('FLAG 1')
plt.ylabel('FLAG 0')
plt.title('PCA Visualization of Data')
plt.colorbar()
plt.show()

from sklearn.preprocessing import MinMaxScaler

# noramalisation process
scale = MinMaxScaler()
x_scaled = scale.fit_transform(x)

print("Shape of x:", x_scaled.shape)
print("Shape of y:", y.shape)
from sklearn.decomposition import PCA

# intialize pca and logistic regression model
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x_scaled)
# Generate and plot a synthetic imbalanced classification dataset
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where

# summarize class distribution
counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(x_pca[row_ix, 0], x_pca[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()
from sklearn.decomposition import PCA

# Assuming you have your data in x and labels in y

# Plotting the distribution of classes in y
unique_classes, class_counts = np.unique(y, return_counts=True)
plt.bar(unique_classes, class_counts)
plt.xlabel('FLAG')
plt.ylabel('COUNT')
plt.title('DATASET DISTRIBUTION')
plt.show()

# Performing PCA to reduce the dimensionality of x for visualization
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x_scaled)

# Plotting the reduced x in 2D
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y, cmap='viridis', alpha=0.5)
plt.xlabel('FLAG 1')
plt.ylabel('FLAG 0')
plt.title('PCA Visualization of Data')
plt.colorbar()
plt.show()

x_pca
# Plotting the reduced x in 2D
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y_resampled, cmap='viridis', alpha=0.5)
plt.xlabel('FLAG 1')
plt.ylabel('FLAG 0')
plt.title('PCA Visualization of Data')
plt.colorbar()
plt.show()

fig, ax = plt.subplots()
ax.scatter(delta1[:-1], delta1[1:], c=close, s=volume, alpha=0.5)

for i in range(0, x_scaled.shape[1]):
    ax.scatter(delta1[:-1], delta1[1:], c=close, s=volume, alpha=0.5)

unique,count=np.unique(y,return_counts=True)
y_dist_val_cnt={k:v for (k,v) in zip(unique,count)}
y_dist_val_cnt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

def smote_with_undersampling(X, y, ratio=1.0):
    # Count of number of samples in each class
    class_counts = Counter(y)

    # Determining the class with fewer samples
    minority_class = min(class_counts, key=class_counts.get)
    majority_class = max(class_counts, key=class_counts.get)

    # Creating SMOTE and Undersampler instances with the specified ratio
    smote = SMOTE(sampling_strategy=ratio, random_state=42)
    undersampler = RandomUnderSampler(sampling_strategy=ratio, random_state=42)

    # Applying SMOTE to create synthetic samples for the minority class
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Applying undersampling to reduce the number of samples in the majority class
    X_resampled, y_resampled = undersampler.fit_resample(X_resampled, y_resampled)

    # Counting the number of samples in each class in the final resampled dataset
    final_class_counts = Counter(y_resampled)

    return X_resampled, y_resampled, final_class_counts

desired_ratio = 1.0
x_resampled, y_resampled, final_class_counts = smote_with_undersampling(x_scaled, y, ratio=desired_ratio)
# x_resampled, y_resampled, final_class_counts = smote_with_undersampling(x, y, ratio=desired_ratio)

# The final_class_counts will now contain equal numbers of samples for both classes
print("Final class counts:", final_class_counts)

unique,count=np.unique(y,return_counts=True)
y_dist_val_cnt={k:v for (k,v) in zip(unique,count)}
y_dist_val_cnt
from sklearn.decomposition import PCA

# Assuming you have your data in x and labels in y

# Plotting the distribution of classes in y
unique_classes, class_counts = np.unique(y_resampled, return_counts=True)
plt.bar(unique_classes, class_counts)
plt.xlabel('FLAG')
plt.ylabel('COUNT')
plt.title('DATASET DISTRIBUTION')
plt.show()

# Performing PCA to reduce the dimensionality of x for visualization
pca = PCA(n_components=2)
x_pca = pca.fit_transform(x_resampled)

# Plotting the reduced x in 2D
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y_resampled, cmap='viridis', alpha=0.5)
plt.xlabel('FLAG 1')
plt.ylabel('FLAG 0')
plt.title('PCA Visualization of Data')
plt.colorbar()
plt.show()

# Generate and plot a synthetic imbalanced classification dataset
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where

# define dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)
# summarize class distribution
counter = Counter(y)
print(counter)
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(y == label)[0]
	pyplot.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
pyplot.legend()
pyplot.show()
X.shape
y.shape
# rawData = pd.read_csv('data/data.csv')

# #data preprocessing 
# #removing column 1 and 2(making InfoData)
# #rawData1_=rawData.iloc[:100,:]
# #rawData2_=rawData.iloc[-100:,:]
# #rawData=pd.concat([rawData1_, rawData2_], ignore_index=True)
# infoData = pd.DataFrame()
# infoData['FLAG'] = rawData['FLAG']
# infoData['CONS_NO'] = rawData['CONS_NO']
# data = rawData.drop(['FLAG', 'CONS_NO'], axis=1)   #axis 1 column ,axis 0 row

# #droping duplicate row
# dropIndex = data[data.duplicated()].index  # duplicates drop
# data = data.drop(dropIndex, axis=0)   #droping duplicate value present wen two row are same
# infoData = infoData.drop(dropIndex, axis=0) #droping duplicate index infodata

# #removing row with all zero(Nan) value
# zeroIndex = data[(data.sum(axis=1) == 0)].index  # zero rows drop
# data = data.drop(zeroIndex, axis=0) 
# infoData = infoData.drop(zeroIndex, axis=0)  

# #change column name to dates(2014/1/1 to 2014-01-01)
# data.columns = pd.to_datetime(data.columns)  #columns reindexing according to dates

# #sort data accoding to date( as previusoly column are unsorted)
# data = data.reindex(sorted(data.columns), axis=1)
# cols = data.columns

# # reindex row name (as some row has been remove till this step due to duplicate or all nan values)
# data.reset_index(inplace=True, drop=True)  # index sorting
# infoData.reset_index(inplace=True, drop=True)

# #filling nan value using neighbouring value (middle missing value replace by average 
# #and other by maximum 2 distance element)
# data = data.interpolate(method='linear', limit=2, limit_direction='both', axis=0).fillna(0) 


# #removing erronoues value(fixing outliers)
# for i in range(data.shape[0]):  # outliers treatment
#     m = data.loc[i].mean()
#     st = data.loc[i].std()
#     data.loc[i] = data.loc[i].mask(data.loc[i] > (m + 3 * st), other=m + 3 * st)

data
infoData
data.to_csv("Data/new_clean_data.csv")
infoData.to_csv("Data/new_infoData.csv")
data.to_csv("../new_clean_data.csv")
infoData.to_csv("../new_infoData.csv")

### 2. Dataset
rawData = pd.read_csv('data/data.csv')

#data preprocessing 
#removing column 1 and 2(making InfoData)
#rawData1_=rawData.iloc[:100,:]
#rawData2_=rawData.iloc[-100:,:]
#rawData=pd.concat([rawData1_, rawData2_], ignore_index=True)
infoData = pd.DataFrame()
infoData['FLAG'] = rawData['FLAG']
infoData['CONS_NO'] = rawData['CONS_NO']
data = rawData.drop(['FLAG', 'CONS_NO'], axis=1)   #axis 1 column ,axis 0 row
data
#droping duplicate row
dropIndex = data[data.duplicated()].index  # duplicates drop
data = data.drop(dropIndex, axis=0)   #droping duplicate value present wen two row are same
infoData = infoData.drop(dropIndex, axis=0) #droping duplicate index infodata
dropIndex
data
#removing row with all zero(Nan) value
zeroIndex = data[(data.sum(axis=1) == 0)].index  # zero rows drop
data = data.drop(zeroIndex, axis=0) 
infoData = infoData.drop(zeroIndex, axis=0)  
zeroIndex
data
#change column name to dates(2014/1/1 to 2014-01-01)
data.columns = pd.to_datetime(data.columns)  #columns reindexing according to dates
data
#sort data accoding to date( as previusoly column are unsorted)
data = data.reindex(sorted(data.columns), axis=1)
cols = data.columns
data
# reindex row name (as some row has been remove till this step due to duplicate or all nan values)
data.reset_index(inplace=True, drop=True)  # index sorting
infoData.reset_index(inplace=True, drop=True)
data
#filling nan value using neighbouring value (middle missing value replace by average 
#and other by maximum 2 distance element)
data = data.interpolate(method='linear', limit=2, limit_direction='both', axis=0).fillna(0) 
data
#removing erronoues value(fixing outliers)
for i in range(data.shape[0]):  # outliers treatment
    m = data.loc[i].mean()
    st = data.loc[i].std()
    data.loc[i] = data.loc[i].mask(data.loc[i] > (m + 3 * st), other=m + 3 * st)

data.loc[i][data.loc[i] > (m + 3 * st)]
data.shape[0]
i = 0
m = data.loc[i].mean()
st = data.loc[i].std()

m, st
data.loc[i].mask(data.loc[i] > (m + 3 * st), other=m + 3 * st)
other=m + 3 * st
other
# Step 1: Import the file
df = pd.read_csv('data.csv')
df.reset_index(inplace=True)
df.set_index('CONS_NO', inplace = True)
df
df.max().max()
df.isnull().sum().sum()
# Extracting 'Flag' column into a new DataFrame
flag_df = df[['FLAG']].copy()
df = df.drop(['index', 'FLAG'], axis = 1)
df
flag_df.head()
# Step 2: Detect and remove rows with more than 600 NaNs
threshold = 600  # Set the threshold for NaN values
rows_to_drop = df[df.isnull().sum(axis=1) > threshold].index  # Get index of rows with more than 600 NaNs

# Step 3: Remove these rows from the DataFrame
df_cleaned = df.drop(index=rows_to_drop)
df_cleaned
df_cleaned.isnull().sum().sum()
# Method 1
# Apply the linear interpolation method
# Assuming df is your DataFrame
df_interpolated = df_cleaned.interpolate(method='linear', axis=1, limit_direction='both')
# Replace NaN values with 0 where interpolation couldn't be applied
df_filled = df_cleaned.fillna(0)

df_filled

df_filled.isnull().sum().sum()
# df_filled.to_csv("Data/clean_data.csv")
# flag_df.to_csv("Data/flag_df.csv")
A = df_filled.values
np.any(A < 0)
A[A < 0]
# Step 1: Import the file
df = pd.read_csv('data/clean_data.csv')
# df.reset_index(inplace=True)
# df.set_index('CONS_NO', inplace = True)
df
df_flag = pd.read_csv('data/flag_df.csv')
df_flag

# # Reattempting the interpolation by rows

# # Apply custom interpolation row-wise
# def interpolate_row(row):
#     for i in range(1, len(row) - 1):  # Iterate over each element in the row, excluding first and last
#         if pd.isnull(row[i]):  # Check if element is NaN
#             if pd.notnull(row[i - 1]) and pd.notnull(row[i + 1]):
#                 row[i] = (row[i - 1] + row[i + 1]) / 2  # Set to average of neighbors
#             else:
#                 row[i] = 0  # Set to 0 if any neighbor is NaN
#     return row

# # Applying the custom interpolation to each row
# df_interpolated = df_cleaned.apply(interpolate_row, axis=1)

# df_interpolated.head()

# # Define a function for the custom interpolation
# def custom_interpolate(series):
#     for i in range(1, len(series) - 1):  # Avoid the first and last elements
#         if pd.isnull(series[i]):  # Check if NaN
#             if pd.notnull(series[i - 1]) and pd.notnull(series[i + 1]):
#                 series[i] = (series[i - 1] + series[i + 1]) / 2  # Compute the mean of the neighbors
#             else:
#                 series[i] = 0  # Set to 0 if neighbors are NaN
#     return series

# # Apply the custom interpolation function to the DataFrame
# df['values'] = custom_interpolate(df['values'])

# df

